In [ ]:
import openai

# LLM API 설정
openai.api_key = "your-api-key"

# 데이터 입력
target_data = {
    "notes": ["C4", "E4", "G4", "A4"],
    "dynamics": [80, 90, 100, 70],
    "pedal": [127, 0, 127, 0]
}

performance_data = {
    "notes": ["C4", "E4", "G4", "F4"],
    "dynamics": [75, 88, 95, 65],
    "pedal": [120, 10, 125, 5]
}

# 프롬프트 생성
prompt = f"""
You are an expert system for evaluating piano performances. Analyze the performance data against the target sheet music and provide the following scores:
1. Note Accuracy: How accurately the performed notes match the target notes, considering time alignment and overlap.
2. Dynamic Similarity: How closely the performed dynamics (e.g., loudness, articulation) resemble the target dynamics.
3. Pedaling Accuracy: How accurately the pedal usage aligns with the target pedal markings.

Input:
- Target Sheet Music Data:
  - Notes: {target_data["notes"]}
  - Dynamics: {target_data["dynamics"]}
  - Pedal: {target_data["pedal"]}
- Performance Data:
  - Notes: {performance_data["notes"]}
  - Dynamics: {performance_data["dynamics"]}
  - Pedal: {performance_data["pedal"]}

Output:
Provide scores for the following:
1. Note Accuracy: (Percentage, reasoning)
2. Dynamic Similarity: (Percentage, reasoning)
3. Pedaling Accuracy: (Percentage, reasoning)
Include overall comments on the performance and areas for improvement.
"""

# API 호출
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=500
)

# 결과 출력
print(response.choices[0].text.strip())
